In [2]:
# import sys
# sys.path.append('../tools')

In [3]:
from fastai.text.all import *
from transformers import (BertTokenizer, AutoModel)
# from train_utils import *
import faiss
from tools.train_utils import *

In [4]:
base_path = 'data'
# base_path = '../data'

BERT_PATH = base_path + '/bert-indo-15g'

bert_model_file = base_path + '/saved_models/text_model/fine_tune_5epoch_bertindo15g-bs-64-mgrad-m06-08.pth'

image_model_file = base_path + '/saved_models/fine_tune_5epoch_nfnetl0-bs-64-pre-mgrad-adaptconcat-embed-1024-m08-10.pth'
OUTPUT_CLASSES = 11014

train_df = add_splits(pd.read_csv(base_path +'/train.csv'))



/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [69]:

    
# class ArcFaceClassifier(nn.Module):
#     def __init__(self, in_features, output_classes):
#         super().__init__()
#         self.initial_layers=nn.Sequential(
#             nn.BatchNorm1d(in_features),
#             nn.Dropout(.25))
#         self.W = nn.Parameter(torch.Tensor(in_features, output_classes))
#         nn.init.kaiming_uniform_(self.W)
#     def forward(self, x):
#         x = self.initial_layers(x)
#         x_norm = F.normalize(x)
#         W_norm = F.normalize(self.W, dim=0)
#         return x_norm @ W_norm
    
class BertArcFace(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = AutoModel.from_pretrained(BERT_PATH)
    def forward(self, x):
        output = self.bert_model(x)
        return output.last_hidden_state[:,0,:]


def load_bert_model(fname):
    model = BertArcFace()
    state = torch.load(fname)
    model.load_state_dict(state)
    return model.cuda().eval()


In [120]:

def neighborhood_search_inference(reference_emb, query_emb, thresh=0.85,  k_neighbors=50):
    
    reference_emb = reference_emb.cpu().numpy()
    query_emb = query_emb.cpu().numpy()
    index = faiss.IndexFlatIP(reference_emb.shape[1])
    
    index.add(reference_emb)
    
    
    sim, idx = index.search(query_emb, k_neighbors)
    sim_thresh = sim.copy()
    idx_thresh = idx.copy()
    reference_thresh = sim[0][0] * thresh
    sim_thresh = sim_thresh[ sim_thresh[sim < reference_thresh]] 
    idx_thresh = idx_thresh[ idx_thresh[sim < reference_thresh]] 

    # sim_thresh[sim[:,1] > min2_thresh,1] = sim[sim[:,1] > min2_thresh,1]
    # idx_thresh[sim[:,1] > min2_thresh,1] = idx[sim[:,1] > min2_thresh,1]
    return idx_thresh, sim_thresh

# def unset_pad(x,pad = -999):
#     x[x == pad] = 0 
#     return x

# def revert_pad(ids, pad = -999):
#     return  np.array([ unset_pad(x,pad) for x in ids],dtype='object')


# def push_all_pad_back(a, pad = -999):
#     valid_mask = a!= pad
#     flipped_mask = valid_mask.sum(1,keepdims=1) > np.arange(a.shape[1]-1,-1,-1)
#     flipped_mask = flipped_mask[:,::-1]
#     a[flipped_mask] = a[valid_mask]
#     a[~flipped_mask] = pad
#     return a


# def join_similarities(join_idx,join_sim,other_idx,other_sim):
#     other_ins = np.array([np.in1d(x,y) for x,y in zip(other_idx,join_idx)])
#     other_sim[other_ins] = 0
#     other_idx[other_ins] = -1
#     union_idx = push_all_pad_back(np.hstack([join_idx,other_idx]), -1)
#     union_sim = push_all_pad_back(np.hstack([join_sim,other_sim]), 0)
#     return union_idx, union_sim

# def join_similarity_set(join_idx,join_sim,img_idx, img_sim,bert_idx, bert_sim):
#     union_idx , union_sim = join_similarities(join_idx, join_sim, img_idx, img_sim)
#     union_idx , union_sim = join_similarities(union_idx, union_sim, bert_idx, bert_sim)
#     return union_idx, union_sim

# def trim_pad(union_idx, union_sim, id_pad=-999):
#     union_idx[union_idx == 0] = id_pad
#     union_idx[union_idx == -1] = 0
#     union_idx = np.array([np.trim_zeros(x,'b') for x in union_idx],dtype='object')
#     union_sim = np.array([np.trim_zeros(x,'b') for x in union_sim],dtype='object')
#     union_idx = revert_pad(union_idx, id_pad)
#     return union_idx, union_sim

# def combine_embeddings(join_emb,img_emb,bert_emb,threshes = [1.0,0.85,0.85]):
#     join_idx, join_sim =  neighborhood_search(join_emb,threshes[0])
#     img_idx, img_sim = neighborhood_search(img_emb,threshes[1])
#     bert_idx, bert_sim =  neighborhood_search(bert_emb,threshes[2])

#     union_idx , union_sim = join_similarities(bert_idx.copy(), bert_sim.copy(), img_idx.copy(), img_sim.copy())
#     # union_idx, union_sim = join_similarity_set(bert_idx.copy(), bert_sim.copy(), img_idx.copy(), img_sim.copy(), join_idx.copy(),join_sim.copy())
#     union_idx, union_sim = trim_pad(union_idx, union_sim)
#     new_emb = blend_neighborhood(join_emb,union_idx,union_sim)
#     # new_emb =None
#     return (img_idx, img_sim), (bert_idx, bert_sim ), (join_idx, join_sim), (union_idx , union_sim), new_emb
    
    
    

# def blend_neighborhood(emb, match_index_lst, similarities_lst):
#     new_emb = emb.clone()
#     for i in tqdm(range(emb.shape[0])):
#         if match_index_lst[i].shape != similarities_lst[i].shape:
#             print(f'data no {i}: id shape {match_index_lst[i].shape} != sim shape {similarities_lst[i].shape} ')
#             break
#         cur_emb = emb[match_index_lst[i]]
        
#         weights = torch.from_numpy(similarities_lst[i]).unsqueeze_(1).cuda()
        
#         new_emb[i] = (cur_emb * weights).sum(axis=0)
        
#     new_emb = F.normalize(new_emb)
    
#     return new_emb


In [70]:
#Taken from https://www.kaggle.com/c/shopee-product-matching/discussion/233605#1278984
def string_escape(s, encoding='utf-8'):
    return s.encode('latin1').decode('unicode-escape').encode('latin1').decode(encoding)

class InferenceTokenizer():
    def __init__(self, path):
        super().__init__()
        self.tokenizer = BertTokenizer.from_pretrained(path)
               
    def encodes(self, text):

        text=string_escape(text)
        encodings = self.tokenizer(text, padding = 'max_length', max_length=100, truncation=True,return_tensors='pt')
        keys =['input_ids', 'attention_mask']
        return tuple(encodings[key].cuda() for key in keys)



In [103]:

model = load_bert_model(bert_model_file)
reference_emb = torch.load('data/saved_embeddings/text_embeddings/fine_tune_5epoch_bertindo15g-bs-64-mgrad-m06-08_embed.pth').cuda()

Some weights of the model checkpoint at data/bert-indo-15g were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [104]:
tokenizer = BertTokenizer.from_pretrained(BERT_PATH)

In [116]:
input_text = 'Paper Bag'

In [117]:
input_text_processed =  torch.tensor([tokenizer.encode(input_text)]).cuda()

In [74]:
input_text_processed.shape

torch.Size([1, 6])

In [118]:


pred_emb = model(input_text_processed)

In [121]:
neighborhood_search_inference(reference_emb, pred_emb.detach(), 0.9)

(array([33161,     0,  7147, 28264, 12255,  1831,  6859, 22569, 25377,
        21149, 11178, 12674, 15606, 26323, 33571, 14768,  8820,  5187,
         9243, 33825, 22570, 22571, 27666, 15836, 12119, 27504, 25819,
        21467, 25034,  4156, 22391, 27649, 23091, 10971,  5667, 15750,
        15811, 19858, 27750, 15378, 16616,  2171, 30008,  9964,   759,
        19197]),
 array([445.90527, 445.90527, 418.6661 , 415.39154, 414.14856, 409.70917,
        409.06158, 407.84808, 405.27185, 401.2542 , 398.88092, 398.0324 ,
        397.9086 , 396.7845 , 396.7685 , 396.5277 , 395.4218 , 395.25696,
        394.75546, 394.23093, 393.02502, 392.64093, 392.33264, 392.09055,
        391.95322, 391.6136 , 391.03336, 390.14285, 389.65686, 389.16006,
        389.02768, 389.02768, 388.92105, 388.83856, 388.29395, 387.50372,
        387.2359 , 386.797  , 386.66592, 386.01196, 385.27756, 384.52478,
        384.22937, 383.9032 , 383.2218 , 382.46735], dtype=float32))